In [3]:
from absbox.report_parser.china import *
import toolz as tz
from lenses import lens
import dateparser
from more_itertools import transpose,unzip

In [4]:
def removePg(xs):
    return [ x for x in xs if not re.match(r"^\d+$",x)]

def removeIfAllinSet(xs:list,s:set):
    return [ x for x in xs if not set(x) == s]

In [55]:
t = '''
计算日 期初本金余额 本金流入 利息流入 期末本金余额
2019/5/1 8,780,266,126.01 - - -
2019/5/31 8,780,266,126.01 84,561,122.98 35,568,680.81 8,695,705,003.03
2019/6/30 8,695,705,003.03 84,811,902.50 35,226,133.34 8,610,893,100.53
2019/7/31 8,610,893,100.53 85,063,707.15 34,882,560.73 8,525,829,393.38
2019/8/31 8,525,829,393.38 85,329,885.29 34,538,016.86 8,440,499,508.09
2019/9/30 8,440,499,508.09 85,505,216.28 34,192,265.10 8,354,994,291.81
2019/10/31 8,354,994,291.81 85,736,614.87 33,845,870.31 8,269,257,676.94
2019/11/30 8,269,257,676.94 85,984,183.69 33,498,533.06 8,183,273,493.25
2019/12/31 8,183,273,493.25 86,197,889.23 33,150,185.86 8,097,075,604.02
2020/1/31 8,097,075,604.02 86,428,457.65 32,800,963.00 8,010,647,146.37
2020/2/29 8,010,647,146.37 86,666,381.76 32,450,797.35 7,923,980,764.61
2020/3/31 7,923,980,764.61 86,902,314.86 32,099,675.39 7,837,078,449.74
2020/4/30 7,837,078,449.74 87,108,580.48 31,747,567.62 7,749,969,869.26
2020/5/31 7,749,969,869.26 87,341,707.64 31,394,623.06 7,662,628,161.62
2020/6/30 7,662,628,161.62 87,486,542.86 31,040,719.75 7,575,141,618.76
2020/7/31 7,575,141,618.76 87,607,806.54 30,686,194.29 7,487,533,812.22
2020/8/31 7,487,533,812.22 87,741,752.06 30,331,166.05 7,399,792,060.16
2020/9/30 7,399,792,060.16 87,927,533.67 29,975,526.60 7,311,864,526.49
2020/10/31 7,311,864,526.49 88,011,627.89 29,619,163.59 7,223,852,898.60
2020/11/30 7,223,852,898.60 88,029,203.92 29,262,399.44 7,135,823,694.69
2020/12/31 7,135,823,694.69 88,120,411.34 28,905,591.13 7,047,703,283.35
2021/1/31 7,047,703,283.35 88,199,153.67 28,548,337.39 6,959,504,129.68
2021/2/28 6,959,504,129.68 88,061,306.10 28,190,636.23 6,871,442,823.57
2021/3/31 6,871,442,823.57 88,147,647.71 27,833,454.50 6,783,295,175.87
2021/4/30 6,783,295,175.87 88,098,444.67 27,475,822.58 6,695,196,731.20
2021/5/31 6,695,196,731.20 88,152,472.75 27,118,496.41 6,607,044,258.44
2021/6/30 6,607,044,258.44 88,159,101.32 26,760,708.90 6,518,885,157.12
2021/7/31 6,518,885,157.12 88,211,657.59 26,403,003.98 6,430,673,499.54
2021/8/31 6,430,673,499.54 88,219,728.04 26,045,060.81 6,342,453,771.50
2021/9/30 6,342,453,771.50 88,273,800.64 25,687,024.80 6,254,179,970.86
2021/10/31 6,254,179,970.86 88,404,798.07 25,329,077.46 6,165,775,172.79
2021/11/30 6,165,775,172.79 88,420,417.45 24,970,056.15 6,077,354,755.34
2021/12/31 6,077,354,755.34 88,343,778.68 24,611,308.06 5,989,010,976.67
2022/1/31 5,989,010,976.67 88,432,944.91 24,252,903.58 5,900,578,031.76
2022/2/28 5,900,578,031.76 88,341,922.51 23,894,113.44 5,812,236,109.25
2022/3/31 5,812,236,109.25 88,422,981.50 23,535,981.04 5,723,813,127.76
39
计算日 期初本金余额 本金流入 利息流入 期末本金余额
2022/4/30 5,723,813,127.76 88,236,459.20 23,177,091.99 5,635,576,668.55
2022/5/31 5,635,576,668.55 88,255,459.61 22,819,554.00 5,547,321,208.94
2022/6/30 5,547,321,208.94 87,988,593.47 22,461,590.70 5,459,332,615.47
2022/7/31 5,459,332,615.47 87,687,228.90 22,104,895.01 5,371,645,386.58
2022/8/31 5,371,645,386.58 87,071,663.95 21,749,213.25 5,284,573,722.62
2022/9/30 5,284,573,722.62 86,408,075.51 21,395,848.27 5,198,165,647.11
2022/10/31 5,198,165,647.11 85,977,284.70 21,045,292.86 5,112,188,362.41
2022/11/30 5,112,188,362.41 85,639,513.05 20,696,195.20 5,026,548,849.36
2022/12/31 5,026,548,849.36 85,495,934.77 20,348,910.37 4,941,052,914.59
2023/1/31 4,941,052,914.59 85,164,861.11 20,001,980.96 4,855,888,053.48
2023/2/28 4,855,888,053.48 84,490,283.02 19,655,836.21 4,771,397,770.46
2023/3/31 4,771,397,770.46 84,153,495.18 19,312,968.97 4,687,244,275.28
2023/4/30 4,687,244,275.28 83,419,427.86 18,970,849.45 4,603,824,847.42
2023/5/31 4,603,824,847.42 83,032,108.78 18,632,175.16 4,520,792,738.65
2023/6/30 4,520,792,738.65 82,551,229.27 18,295,364.52 4,438,241,509.38
2023/7/31 4,438,241,509.38 81,809,259.35 17,959,490.88 4,356,432,250.03
2023/8/31 4,356,432,250.03 81,397,177.19 17,627,176.38 4,275,035,072.84
2023/9/30 4,275,035,072.84 81,139,521.58 17,296,499.13 4,193,895,551.26
2023/10/31 4,193,895,551.26 80,828,072.05 16,967,241.39 4,113,067,479.21
2023/11/30 4,113,067,479.21 80,542,366.27 16,638,918.76 4,032,525,112.94
2023/12/31 4,032,525,112.94 80,146,964.45 16,312,251.16 3,952,378,148.49
2024/1/31 3,952,378,148.49 79,495,085.58 15,986,799.77 3,872,883,062.91
2024/2/29 3,872,883,062.91 78,700,432.60 15,664,545.21 3,794,182,630.30
2024/3/31 3,794,182,630.30 78,161,163.65 15,345,323.09 3,716,021,466.65
2024/4/30 3,716,021,466.65 77,495,662.39 15,027,980.86 3,638,525,804.26
2024/5/31 3,638,525,804.26 77,091,991.12 14,714,061.86 3,561,433,813.14
2024/6/30 3,561,433,813.14 76,801,634.22 14,401,942.46 3,484,632,178.92
2024/7/31 3,484,632,178.92 76,090,844.18 14,090,844.57 3,408,541,334.75
2024/8/31 3,408,541,334.75 75,550,694.75 13,782,811.43 3,332,990,640.00
2024/9/30 3,332,990,640.00 75,024,912.30 13,477,253.56 3,257,965,727.70
2024/10/31 3,257,965,727.70 74,270,625.45 13,173,974.71 3,183,695,102.25
2024/11/30 3,183,695,102.25 73,786,538.37 12,874,083.83 3,109,908,563.87
2024/12/31 3,109,908,563.87 73,398,362.25 12,576,502.66 3,036,510,201.62
2025/1/31 3,036,510,201.62 72,967,953.39 12,280,192.37 2,963,542,248.23
2025/2/28 2,963,542,248.23 71,984,873.43 11,985,703.92 2,891,557,374.80
2025/3/31 2,891,557,374.80 71,331,070.40 11,695,806.88 2,820,226,304.40
2025/4/30 2,820,226,304.40 69,860,964.26 11,407,913.05 2,750,365,340.14
2025/5/31 2,750,365,340.14 68,971,605.34 11,126,739.19 2,681,393,734.80
2025/6/30 2,681,393,734.80 67,946,691.98 10,848,759.74 2,613,447,042.82
2025/7/31 2,613,447,042.82 66,185,543.52 10,575,241.12 2,547,261,499.30
2025/8/31 2,547,261,499.30 65,040,378.41 10,308,918.84 2,482,221,120.89
2025/9/30 2,482,221,120.89 63,490,874.91 10,046,557.99 2,418,730,245.98
2025/10/31 2,418,730,245.98 62,153,176.80 9,790,938.88 2,356,577,069.18
2025/11/30 2,356,577,069.18 60,973,089.25 9,540,817.86 2,295,603,979.93
2025/12/31 2,295,603,979.93 59,635,282.76 9,295,540.30 2,235,968,697.18
2026/1/31 2,235,968,697.18 58,414,456.34 9,055,518.17 2,177,554,240.84
40
计算日 期初本金余额 本金流入 利息流入 期末本金余额
2026/2/28 2,177,554,240.84 56,760,165.30 8,820,241.00 2,120,794,075.54
2026/3/31 2,120,794,075.54 56,172,628.81 8,591,654.12 2,064,621,446.73
2026/4/30 2,064,621,446.73 55,064,595.30 8,365,370.20 2,009,556,851.42
2026/5/31 2,009,556,851.42 54,261,248.51 8,143,997.84 1,955,295,602.91
2026/6/30 1,955,295,602.91 53,387,330.63 7,925,088.75 1,901,908,272.28
2026/7/31 1,901,908,272.28 51,836,105.80 7,710,085.16 1,850,072,166.48
2026/8/31 1,850,072,166.48 50,651,516.04 7,501,298.07 1,799,420,650.44
2026/9/30 1,799,420,650.44 49,476,281.49 7,296,462.94 1,749,944,368.95
2026/10/31 1,749,944,368.95 48,369,862.39 7,097,072.95 1,701,574,506.56
2026/11/30 1,701,574,506.56 47,494,525.57 6,901,125.55 1,654,079,980.99
2026/12/31 1,654,079,980.99 46,259,545.19 6,709,035.17 1,607,820,435.80
2027/1/31 1,607,820,435.80 45,469,077.54 6,521,612.93 1,562,351,358.26
2027/2/28 1,562,351,358.26 44,112,169.49 6,337,148.84 1,518,239,188.76
2027/3/31 1,518,239,188.76 43,575,620.70 6,158,370.30 1,474,663,568.06
2027/4/30 1,474,663,568.06 42,285,815.12 5,981,097.43 1,432,377,752.94
2027/5/31 1,432,377,752.94 41,680,807.99 5,809,542.45 1,390,696,944.94
2027/6/30 1,390,696,944.94 40,470,776.93 5,639,429.08 1,350,226,168.02
2027/7/31 1,350,226,168.02 39,214,171.97 5,474,159.19 1,311,011,996.05
2027/8/31 1,311,011,996.05 37,962,508.94 5,313,781.08 1,273,049,487.11
2027/9/30 1,273,049,487.11 36,993,695.82 5,158,734.93 1,236,055,791.29
2027/10/31 1,236,055,791.29 35,817,969.49 5,006,640.10 1,200,237,821.79
2027/11/30 1,200,237,821.79 34,934,258.64 4,860,122.39 1,165,303,563.16
2027/12/31 1,165,303,563.16 34,348,855.10 4,717,653.71 1,130,954,708.06
2028/1/31 1,130,954,708.06 33,333,150.65 4,575,698.25 1,097,621,557.41
2028/2/29 1,097,621,557.41 32,213,906.42 4,438,509.91 1,065,407,650.99
2028/3/31 1,065,407,650.99 31,843,843.01 4,306,759.35 1,033,563,807.98
2028/4/30 1,033,563,807.98 30,993,462.79 4,175,520.32 1,002,570,345.19
2028/5/31 1,002,570,345.19 30,353,792.40 4,048,896.62 972,216,552.78
2028/6/30 972,216,552.78 29,322,341.62 3,923,950.72 942,894,211.16
2028/7/31 942,894,211.16 28,938,979.14 3,804,054.99 913,955,232.02
2028/8/31 913,955,232.02 27,938,398.77 3,684,405.76 886,016,833.25
2028/9/30 886,016,833.25 27,524,103.82 3,569,867.70 858,492,729.43
2028/10/31 858,492,729.43 27,084,462.26 3,457,689.25 831,408,267.17
2028/11/30 831,408,267.17 26,158,139.18 3,345,687.34 805,250,127.99
2028/12/31 805,250,127.99 25,412,571.27 3,238,680.90 779,837,556.72
2029/1/31 779,837,556.72 24,735,358.78 3,134,956.76 755,102,197.94
2029/2/28 755,102,197.94 23,956,223.14 3,034,054.01 731,145,974.79
2029/3/31 731,145,974.79 23,447,607.32 2,936,711.33 707,698,367.48
2029/4/30 707,698,367.48 22,772,830.84 2,840,725.22 684,925,536.63
2029/5/31 684,925,536.63 22,132,798.93 2,748,084.07 662,792,737.70
2029/6/30 662,792,737.70 21,613,311.23 2,658,211.77 641,179,426.47
2029/7/31 641,179,426.47 21,094,249.04 2,570,303.44 620,085,177.43
2029/8/31 620,085,177.43 20,397,139.06 2,484,530.81 599,688,038.37
2029/9/30 599,688,038.37 19,840,650.93 2,401,661.50 579,847,387.44
2029/10/31 579,847,387.44 19,280,034.74 2,321,318.47 560,567,352.71
2029/11/30 560,567,352.71 19,039,048.65 2,243,454.91 541,528,304.06
41
计算日 期初本金余额 本金流入 利息流入 期末本金余额
2029/12/31 541,528,304.06 18,755,466.89 2,167,469.78 522,772,837.17
2030/1/31 522,772,837.17 18,013,338.62 2,091,309.41 504,759,498.55
2030/2/28 504,759,498.55 17,004,870.08 2,018,314.53 487,754,628.47
2030/3/31 487,754,628.47 16,507,347.02 1,950,151.41 471,247,281.45
2030/4/30 471,247,281.45 15,528,239.22 1,883,445.31 455,719,042.23
2030/5/31 455,719,042.23 14,870,891.84 1,821,089.99 440,848,150.39
2030/6/30 440,848,150.39 14,148,820.89 1,761,227.06 426,699,329.51
2030/7/31 426,699,329.51 13,354,808.57 1,704,210.20 413,344,520.94
2030/8/31 413,344,520.94 13,094,290.03 1,650,683.17 400,250,230.91
2030/9/30 400,250,230.91 12,847,743.04 1,597,718.23 387,402,487.87
2030/10/31 387,402,487.87 12,433,893.61 1,545,927.94 374,968,594.26
2030/11/30 374,968,594.26 12,273,752.12 1,495,828.46 362,694,842.15
2030/12/31 362,694,842.15 12,156,623.81 1,446,468.69 350,538,218.34
2031/1/31 350,538,218.34 11,942,511.41 1,397,582.86 338,595,706.92
2031/2/28 338,595,706.92 11,535,016.29 1,349,228.69 327,060,690.64
2031/3/31 327,060,690.64 11,393,187.22 1,302,710.10 315,667,503.42
2031/4/30 315,667,503.42 11,156,347.31 1,256,607.22 304,511,156.11
2031/5/31 304,511,156.11 10,932,441.62 1,211,450.14 293,578,714.49
2031/6/30 293,578,714.49 10,759,790.93 1,167,121.74 282,818,923.56
2031/7/31 282,818,923.56 10,471,159.14 1,123,557.14 272,347,764.42
2031/8/31 272,347,764.42 10,153,547.08 1,081,010.49 262,194,217.34
2031/9/30 262,194,217.34 9,830,676.84 1,039,779.34 252,363,540.50
2031/10/31 252,363,540.50 9,594,819.43 999,862.58 242,768,721.07
2031/11/30 242,768,721.07 9,393,041.10 960,770.48 233,375,679.97
2031/12/31 233,375,679.97 9,195,199.00 922,583.37 224,180,480.97
2032/1/31 224,180,480.97 9,045,061.48 885,122.04 215,135,419.49
2032/2/29 215,135,419.49 8,789,051.23 848,324.91 206,346,368.26
2032/3/31 206,346,368.26 8,717,896.21 812,766.70 197,628,472.05
2032/4/30 197,628,472.05 8,419,329.32 777,053.07 189,209,142.73
2032/5/31 189,209,142.73 8,222,109.58 742,958.16 180,987,033.15
2032/6/30 180,987,033.15 7,968,683.00 709,409.92 173,018,350.15
2032/7/31 173,018,350.15 7,710,968.31 676,920.55 165,307,381.83
2032/8/31 165,307,381.83 7,348,341.72 645,293.47 157,959,040.11
2032/9/30 157,959,040.11 7,157,051.70 615,201.94 150,801,988.41
2032/10/31 150,801,988.41 7,014,841.57 586,120.31 143,787,146.84
2032/11/30 143,787,146.84 6,754,093.39 557,403.61 137,033,053.45
2032/12/31 137,033,053.45 6,403,595.26 529,770.21 130,629,458.19
2033/1/31 130,629,458.19 6,218,070.23 503,697.60 124,411,387.96
2033/2/28 124,411,387.96 5,843,239.90 477,952.59 118,568,148.06
2033/3/31 118,568,148.06 5,697,296.25 454,135.97 112,870,851.81
2033/4/30 112,870,851.81 5,355,237.76 430,913.31 107,515,614.05
2033/5/31 107,515,614.05 5,116,479.17 409,559.22 102,399,134.89
2033/6/30 102,399,134.89 4,768,695.28 388,637.90 97,630,439.61
2033/7/31 97,630,439.61 4,513,191.31 369,552.73 93,117,248.30
2033/8/31 93,117,248.30 3,977,748.16 350,870.95 89,139,500.14
2033/9/30 89,139,500.14 3,579,731.46 334,853.77 85,559,768.68
42
计算日 期初本金余额 本金流入 利息流入 期末本金余额
2033/10/31 85,559,768.68 3,331,794.58 320,597.16 82,227,974.10
2033/11/30 82,227,974.10 3,153,170.14 307,186.62 79,074,803.96
2033/12/31 79,074,803.96 3,070,346.50 294,646.99 76,004,457.46
2034/1/31 76,004,457.46 3,026,154.90 282,451.82 72,978,302.56
2034/2/28 72,978,302.56 2,727,827.75 270,477.19 70,250,474.81
2034/3/31 70,250,474.81 2,494,021.13 259,811.97 67,756,453.68
2034/4/30 67,756,453.68 2,257,547.74 250,173.65 65,498,905.93
2034/5/31 65,498,905.93 2,199,453.11 241,584.79 63,299,452.83
2034/6/30 63,299,452.83 2,183,553.03 233,259.71 61,115,899.79
2034/7/31 61,115,899.79 2,154,789.33 224,962.80 58,961,110.46
2034/8/31 58,961,110.46 2,125,870.53 216,830.10 56,835,239.93
2034/9/30 56,835,239.93 2,081,109.09 208,759.39 54,754,130.84
2034/10/31 54,754,130.84 2,008,429.14 200,922.44 52,745,701.70
2034/11/30 52,745,701.70 1,969,915.05 193,408.85 50,775,786.65
2034/12/31 50,775,786.65 1,983,853.86 186,139.06 48,791,932.79
2035/1/31 48,791,932.79 1,932,995.66 178,748.41 46,858,937.13
2035/2/28 46,858,937.13 1,858,156.42 171,436.98 45,000,780.71
2035/3/31 45,000,780.71 1,860,570.70 164,605.63 43,140,210.02
2035/4/30 43,140,210.02 1,828,877.08 157,670.92 41,311,332.94
2035/5/31 41,311,332.94 1,805,983.22 150,906.94 39,505,349.72
2035/6/30 39,505,349.72 1,798,211.47 144,233.04 37,707,138.25
2035/7/31 37,707,138.25 1,789,413.22 137,588.03 35,917,725.03
2035/8/31 35,917,725.03 1,786,363.67 130,978.11 34,131,361.36
2035/9/30 34,131,361.36 1,778,833.18 124,381.56 32,352,528.18
2035/10/31 32,352,528.18 1,743,420.53 117,815.27 30,609,107.65
2035/11/30 30,609,107.65 1,677,758.38 111,383.82 28,931,349.27
2035/12/31 28,931,349.27 1,600,185.35 105,202.61 27,331,163.92
2036/1/31 27,331,163.92 1,570,396.27 99,320.64 25,760,767.65
2036/2/29 25,760,767.65 1,524,325.92 93,552.38 24,236,441.73
2036/3/31 24,236,441.73 1,489,684.83 87,970.62 22,746,756.90
2036/4/30 22,746,756.90 1,410,479.73 82,478.41 21,336,277.17
2036/5/31 21,336,277.17 1,360,624.37 77,311.88 19,975,652.80
2036/6/30 19,975,652.80 1,327,292.62 72,315.31 18,648,360.17
2036/7/31 18,648,360.17 1,284,845.70 67,449.60 17,363,514.48
2036/8/31 17,363,514.48 1,235,877.03 62,758.03 16,127,637.44
2036/9/30 16,127,637.44 1,147,961.46 58,212.04 14,979,675.99
2036/10/31 14,979,675.99 1,092,438.51 54,016.40 13,887,237.47
2036/11/30 13,887,237.47 991,593.73 49,999.49 12,895,643.75
2036/12/31 12,895,643.75 901,719.78 46,370.50 11,993,923.96
2037/1/31 11,993,923.96 793,977.42 43,066.17 11,199,946.54
2037/2/28 11,199,946.54 682,544.75 40,162.61 10,517,401.79
2037/3/31 10,517,401.79 582,704.74 37,652.36 9,934,697.05
2037/4/30 9,934,697.05 532,523.80 35,490.91 9,402,173.25
2037/5/31 9,402,173.25 518,173.99 33,593.70 8,883,999.26
2037/6/30 8,883,999.26 396,497.75 31,610.05 8,487,501.51
2037/7/31 8,487,501.51 348,400.78 30,144.37 8,139,100.73
43
计算日 期初本金余额 本金流入 利息流入 期末本金余额
2037/8/31 8,139,100.73 327,737.90 28,866.34 7,811,362.84
2037/9/30 7,811,362.84 314,281.89 27,665.29 7,497,080.95
2037/10/31 7,497,080.95 298,661.23 26,518.72 7,198,419.71
2037/11/30 7,198,419.71 289,579.14 25,436.57 6,908,840.57
2037/12/31 6,908,840.57 282,777.86 24,387.77 6,626,062.71
2038/1/31 6,626,062.71 276,193.85 23,368.00 6,349,868.86
2038/2/28 6,349,868.86 265,411.52 22,377.76 6,084,457.34
2038/3/31 6,084,457.34 257,873.95 21,439.43 5,826,583.40
2038/4/30 5,826,583.40 252,369.34 20,511.36 5,574,214.05
2038/5/31 5,574,214.05 244,047.91 19,613.33 5,330,166.14
2038/6/30 5,330,166.14 244,866.03 18,744.80 5,085,300.10
2038/7/31 5,085,300.10 245,687.07 17,873.35 4,839,613.04
2038/8/31 4,839,613.04 246,511.04 16,998.97 4,593,102.00
2038/9/30 4,593,102.00 246,585.96 16,121.65 4,346,516.04
2038/10/31 4,346,516.04 247,413.36 15,243.83 4,099,102.68
2038/11/30 4,099,102.68 230,530.68 14,363.06 3,868,572.00
2038/12/31 3,868,572.00 227,386.36 13,546.87 3,641,185.64
2039/1/31 3,641,185.64 217,578.93 12,742.32 3,423,606.71
2039/2/28 3,423,606.71 205,792.76 11,974.51 3,217,813.96
2039/3/31 3,217,813.96 205,335.45 11,249.00 3,012,478.51
2039/4/30 3,012,478.51 200,032.60 10,524.80 2,812,445.91
2039/5/31 2,812,445.91 192,855.13 9,820.73 2,619,590.78
2039/6/30 2,619,590.78 190,717.95 9,142.34 2,428,872.84
2039/7/31 2,428,872.84 191,348.13 8,471.91 2,237,524.71
2039/8/31 2,237,524.71 188,598.12 7,799.25 2,048,926.59
2039/9/30 2,048,926.59 182,231.29 7,136.11 1,866,695.30
2039/10/31 1,866,695.30 182,838.34 6,496.45 1,683,856.96
2039/11/30 1,683,856.96 161,461.85 5,854.66 1,522,395.11
2039/12/31 1,522,395.11 157,142.61 5,287.03 1,365,252.50
2040/1/31 1,365,252.50 145,543.08 4,735.65 1,219,709.42
2040/2/29 1,219,709.42 145,425.77 4,228.04 1,074,283.65
2040/3/31 1,074,283.65 142,162.78 3,720.83 932,120.87
2040/4/30 932,120.87 132,082.92 3,225.69 800,037.95
2040/5/31 800,037.95 126,556.15 2,767.60 673,481.80
2040/6/30 673,481.80 122,332.76 2,329.92 551,149.05
2040/7/31 551,149.05 112,605.53 1,906.94 438,543.51
2040/8/31 438,543.51 109,409.95 1,517.15 329,133.57
2040/9/30 329,133.57 104,747.52 1,137.72 224,386.04
2040/10/31 224,386.04 87,641.06 774.18 136,744.99
2040/11/30 136,744.99 78,980.65 471.33 57,764.33
2040/12/31 57,764.33 57,764.33 199.28 0.00

'''

In [56]:
tz.pipe(t
       ,removeComma
       ,lambda x: x.strip().split("\n")
       ,removePg
       ,splitSpace
       ,lambda xs: xs[2:]
       ,lambda xs: removeIfAllinSet(xs, set(['计算日', '期初本金余额', '本金流入', '利息流入', '期末本金余额']))
       ,lambda xs: xs & lens.Each()[0].modify(lambda y:dateparser.parse(y))  
       ,lambda xs: xs & lens.Each()[0].modify(lambda y: y.strftime("%Y-%m-%d")) 
       ,lambda xs: xs & lens.Each().Fork(lens[1],lens[2],lens[3],lens[4]).modify(float)
       ,lambda xs: list(transpose(xs))
       ,lambda xs: list(transpose([xs[0],xs[4],xs[2],xs[3]]))
       ,lambda xs: xs & lens.Each().modify(list)
       )

[['2019-05-31', 8695705003.03, 84561122.98, 35568680.81],
 ['2019-06-30', 8610893100.53, 84811902.5, 35226133.34],
 ['2019-07-31', 8525829393.38, 85063707.15, 34882560.73],
 ['2019-08-31', 8440499508.09, 85329885.29, 34538016.86],
 ['2019-09-30', 8354994291.81, 85505216.28, 34192265.1],
 ['2019-10-31', 8269257676.94, 85736614.87, 33845870.31],
 ['2019-11-30', 8183273493.25, 85984183.69, 33498533.06],
 ['2019-12-31', 8097075604.02, 86197889.23, 33150185.86],
 ['2020-01-31', 8010647146.37, 86428457.65, 32800963.0],
 ['2020-02-29', 7923980764.61, 86666381.76, 32450797.35],
 ['2020-03-31', 7837078449.74, 86902314.86, 32099675.39],
 ['2020-04-30', 7749969869.26, 87108580.48, 31747567.62],
 ['2020-05-31', 7662628161.62, 87341707.64, 31394623.06],
 ['2020-06-30', 7575141618.76, 87486542.86, 31040719.75],
 ['2020-07-31', 7487533812.22, 87607806.54, 30686194.29],
 ['2020-08-31', 7399792060.16, 87741752.06, 30331166.05],
 ['2020-09-30', 7311864526.49, 87927533.67, 29975526.6],
 ['2020-10-31', 72

In [86]:
bs = '''
支付日 优先A-2档目标余额 支付日 优先A-2档目标余额
2019/11/26 4,228,500,000 2022/4/26 2,087,000,000
2019/12/26 4,156,000,000 2022/5/26 1,989,500,000
2020/1/26 4,083,500,000 2022/6/26 1,894,500,000
2020/2/26 4,011,000,000 2022/7/26 1,799,500,000
2020/3/26 3,938,500,000 2022/8/26 1,709,500,000
2020/4/26 3,868,500,000 2022/9/26 1,622,000,000
2020/5/26 3,798,500,000 2022/10/26 1,534,500,000
2020/6/26 3,728,500,000 2022/11/26 1,447,000,000
2020/7/26 3,658,500,000 2022/12/26 1,359,500,000
2020/8/26 3,593,500,000 2023/1/26 1,272,000,000
2020/9/26 3,528,500,000 2023/2/26 1,187,000,000
2020/10/26 3,463,500,000 2023/3/26 1,102,000,000
2020/11/26 3,398,500,000 2023/4/26 1,017,000,000
2020/12/26 3,333,500,000 2023/5/26 937,000,000
2021/1/26 3,271,000,000 2023/6/26 862,000,000
2021/2/26 3,208,500,000 2023/7/26 787,000,000
2021/3/26 3,146,000,000 2023/8/26 712,000,000
2021/4/26 3,083,500,000 2023/9/26 637,000,000
2021/5/26 3,021,000,000 2023/10/26 562,000,000
2021/6/26 2,963,000,000 2023/11/26 492,000,000
2021/7/26 2,905,000,000 2023/12/26 422,000,000
2021/8/26 2,847,000,000 2024/1/26 352,000,000
2021/9/26 2,754,500,000 2024/2/26 282,000,000
2021/10/26 2,662,000,000 2024/3/26 214,500,000
2021/11/26 2,569,500,000 2024/4/26 147,000,000
2021/12/26 2,477,000,000 2024/5/26 82,000,000
2022/1/26 2,379,500,000 2024/6/26 17,000,000
2022/2/26 2,282,000,000 2024/7/26 0
2022/3/26 2,184,500,000 
'''

#{"PAC":
#   [["2021-07-20",800]
#   ,["2021-08-20",710]
#   ,["2021-09-20",630]
#   ,["2021-10-20",0]
#   ]}

In [70]:
tz.pipe(bs
       ,removeComma
       ,lambda x: x.strip().split("\n")
       ,splitSpace
       ,lambda xs: xs[1:]
       ,lambda xs: xs & lens.Each().modify(lambda y: list(tz.partition(2,y)))
       ,lambda xs: list(transpose(xs))
       #,lambda xs : list(tz.concat(xs))
       #,lambda xs : xs & lens.Each()[1].modify(float) 
       #,lambda xs : xs & lens.Each()[0].modify(lambda y: dateparser.parse(y).strftime("%Y-%m-%d"))  
       #,lambda xs : xs & lens.Each().modify(list) 
       )

[(('2019/11/26', '955000000'),
  ('2019/12/26', '910000000'),
  ('2020/1/26', '865000000'),
  ('2020/2/26', '820000000'),
  ('2020/3/26', '775000000'),
  ('2020/4/26', '730000000'),
  ('2020/5/26', '685000000'),
  ('2020/6/26', '640000000'),
  ('2020/7/26', '595000000'),
  ('2020/8/26', '550000000'),
  ('2020/9/26', '505000000'),
  ('2020/10/26', '460000000'))]

In [87]:
tz.pipe(bs
       ,removeComma
       ,lambda x: x.strip().split("\n")
       ,splitSpace
       ,lambda xs: xs[1:]
       ,lambda xs: xs & lens.Each().modify(lambda y: list(tz.partition(2,y)))
       ,lambda xs: unzip(xs)
       ,lambda xs: (list(xs[0]),list(xs[1]))
       ,lambda xs: list(tz.concat(xs))
       
       ,lambda xs : xs & lens.Each()[1].modify(float) 
       ,lambda xs : xs & lens.Each()[0].modify(lambda y: dateparser.parse(y).strftime("%Y-%m-%d"))  
       ,lambda xs : xs & lens.Each().modify(list) 
       )

[['2019-11-26', 4228500000.0],
 ['2019-12-26', 4156000000.0],
 ['2020-01-26', 4083500000.0],
 ['2020-02-26', 4011000000.0],
 ['2020-03-26', 3938500000.0],
 ['2020-04-26', 3868500000.0],
 ['2020-05-26', 3798500000.0],
 ['2020-06-26', 3728500000.0],
 ['2020-07-26', 3658500000.0],
 ['2020-08-26', 3593500000.0],
 ['2020-09-26', 3528500000.0],
 ['2020-10-26', 3463500000.0],
 ['2020-11-26', 3398500000.0],
 ['2020-12-26', 3333500000.0],
 ['2021-01-26', 3271000000.0],
 ['2021-02-26', 3208500000.0],
 ['2021-03-26', 3146000000.0],
 ['2021-04-26', 3083500000.0],
 ['2021-05-26', 3021000000.0],
 ['2021-06-26', 2963000000.0],
 ['2021-07-26', 2905000000.0],
 ['2021-08-26', 2847000000.0],
 ['2021-09-26', 2754500000.0],
 ['2021-10-26', 2662000000.0],
 ['2021-11-26', 2569500000.0],
 ['2021-12-26', 2477000000.0],
 ['2022-01-26', 2379500000.0],
 ['2022-02-26', 2282000000.0],
 ['2022-03-26', 2184500000.0],
 ['2022-04-26', 2087000000.0],
 ['2022-05-26', 1989500000.0],
 ['2022-06-26', 1894500000.0],
 ['2022-

In [88]:
t = '''
2024/5/31 4,149,993,508.70 9,489,167.94 172,490,525.44 3,977,502,983.26
2024/6/30 3,977,502,983.26 9,279,237.67 172,700,455.71 3,804,802,527.55
2024/7/31 3,804,802,527.55 9,068,128.92 172,911,564.46 3,631,890,963.10
2024/8/31 3,631,890,963.10 8,855,834.39 173,123,858.99 3,458,767,104.11
2024/9/30 3,458,767,104.11 8,642,346.73 173,337,346.65 3,285,429,757.46
2024/10/31 3,285,429,757.46 8,427,658.54 173,552,034.83 3,111,877,722.63
2024/11/30 3,111,877,722.63 8,211,762.39 173,767,930.99 2,938,109,791.64
2024/12/31 2,938,109,791.64 7,994,650.77 173,985,042.61 2,764,124,749.02
2025/1/31 2,764,124,749.02 7,776,316.13 170,336,781.46 2,593,787,967.56
2025/2/28 2,593,787,967.56 7,557,092.96 167,205,209.74 2,426,582,757.82
2025/3/31 2,426,582,757.82 7,337,105.08 165,572,906.02 2,261,009,851.80
2025/4/30 2,261,009,851.80 7,116,695.09 163,435,679.31 2,097,574,172.49
2025/5/31 2,097,574,172.49 6,896,013.68 161,400,476.24 1,936,173,696.25
2025/6/30 1,936,173,696.25 6,675,129.84 159,397,408.48 1,776,776,287.77
2025/7/31 1,776,776,287.77 6,454,115.49 146,782,706.75 1,629,993,581.02
2025/8/31 1,629,993,581.02 6,234,935.04 134,147,202.96 1,495,846,378.06
2025/9/30 1,495,846,378.06 6,018,111.36 123,350,083.25 1,372,496,294.80
2025/10/31 1,372,496,294.80 5,802,752.67 112,626,976.42 1,259,869,318.38
2025/11/30 1,259,869,318.38 5,588,008.05 101,245,049.72 1,158,624,268.66
2025/12/31 1,158,624,268.66 5,374,025.59 91,103,350.41 1,067,520,918.25
2026/1/31 1,067,520,918.25 5,161,537.07 77,836,918.08 989,684,000.17
2026/2/28 989,684,000.17 4,950,800.06 66,283,227.67 923,400,772.51
2026/3/31 923,400,772.51 4,741,336.86 60,675,927.24 862,724,845.27
2026/4/30 862,724,845.27 4,532,222.30 59,801,932.76 802,922,912.50
2026/5/31 802,922,912.50 4,324,564.05 58,607,600.57 744,315,311.93
2026/6/30 744,315,311.93 4,119,534.47 57,544,517.04 686,770,794.90
2026/7/31 686,770,794.90 3,916,430.55 52,321,906.10 634,448,888.80
2026/8/31 634,448,888.80 3,725,836.98 47,383,542.69 587,065,346.11
2026/9/30 587,065,346.11 3,545,710.11 42,666,049.07 544,399,297.04
2026/10/31 544,399,297.04 3,374,257.30 38,214,011.37 506,185,285.67
2026/11/30 506,185,285.67 3,211,467.81 34,186,862.28 471,998,423.39
2026/12/31 471,998,423.39 3,055,049.75 30,831,477.85 441,166,945.54
2027/1/31 441,166,945.54 2,903,665.98 26,898,405.53 414,268,540.01
2027/2/28 414,268,540.01 2,757,713.31 23,120,571.29 391,147,968.72
2027/3/31 391,147,968.72 2,617,485.64 21,243,568.63 369,904,400.09
2027/4/30 369,904,400.09 2,479,263.62 21,310,218.04 348,594,182.05
2027/5/31 348,594,182.05 2,340,456.30 21,349,492.79 327,244,689.26
2027/6/30 327,244,689.26 2,201,186.66 21,413,919.75 305,830,769.51
2027/7/31 305,830,769.51 2,061,327.89 21,244,995.11 284,585,774.40
2027/8/31 284,585,774.40 1,922,073.93 21,066,757.50 263,519,016.90
2027/9/30 263,519,016.90 1,783,472.90 20,943,557.87 242,575,459.03
2027/10/31 242,575,459.03 1,645,233.74 20,794,245.34 221,781,213.69
2027/11/30 221,781,213.69 1,507,585.84 20,609,713.72 201,171,499.97
2027/12/31 201,171,499.97 1,370,616.83 20,423,516.78 180,747,983.19
2028/1/31 180,747,983.19 1,234,388.59 20,186,895.08 160,561,088.11
2028/2/29 160,561,088.11 1,099,179.68 19,836,254.95 140,724,833.16
2028/3/31 140,724,833.16 965,743.33 19,511,463.81 121,213,369.35
2028/4/30 121,213,369.35 833,929.46 19,217,486.74 101,995,882.61
2028/5/31 101,995,882.61 703,883.48 19,006,705.89 82,989,176.72
2028/6/30 82,989,176.72 575,036.48 17,548,869.73 65,440,307.00
2028/7/31 65,440,307.00 455,181.32 15,854,306.25 49,586,000.75
2028/8/31 49,586,000.75 345,896.15 14,021,715.77 35,564,284.98
2028/9/30 35,564,284.98 248,781.42 12,021,315.21 23,542,969.76
2028/10/31 23,542,969.76 165,336.02 9,804,734.48 13,738,235.28
2028/11/30 13,738,235.28 97,000.92 7,540,710.90 6,197,524.38
2028/12/31 6,197,524.38 44,184.74 4,568,328.13 1,629,196.25
2029/1/31 1,629,196.25 11,802.14 1,629,196.25 0.00
'''

In [107]:
tz.pipe(t
       ,removeComma
       ,lambda x: x.strip().split("\n")
       ,splitSpace
       ,lambda xs: xs & lens.Each()[0].modify(lambda y:dateparser.parse(y))  
       ,lambda xs: xs & lens.Each()[0].modify(lambda y: y.strftime("%Y-%m-%d")) 
       ,lambda xs: xs & lens.Each().Fork(lens[1],lens[2],lens[3],lens[4]).modify(float)
       ,lambda xs: list(transpose(xs))
       ,lambda xs: list(transpose([xs[0],xs[4],xs[3],xs[2]]))
       ,lambda xs: xs & lens.Each().modify(list)
       )

[['2024-05-31', 3977502983.26, 172490525.44, 9489167.94],
 ['2024-06-30', 3804802527.55, 172700455.71, 9279237.67],
 ['2024-07-31', 3631890963.1, 172911564.46, 9068128.92],
 ['2024-08-31', 3458767104.11, 173123858.99, 8855834.39],
 ['2024-09-30', 3285429757.46, 173337346.65, 8642346.73],
 ['2024-10-31', 3111877722.63, 173552034.83, 8427658.54],
 ['2024-11-30', 2938109791.64, 173767930.99, 8211762.39],
 ['2024-12-31', 2764124749.02, 173985042.61, 7994650.77],
 ['2025-01-31', 2593787967.56, 170336781.46, 7776316.13],
 ['2025-02-28', 2426582757.82, 167205209.74, 7557092.96],
 ['2025-03-31', 2261009851.8, 165572906.02, 7337105.08],
 ['2025-04-30', 2097574172.49, 163435679.31, 7116695.09],
 ['2025-05-31', 1936173696.25, 161400476.24, 6896013.68],
 ['2025-06-30', 1776776287.77, 159397408.48, 6675129.84],
 ['2025-07-31', 1629993581.02, 146782706.75, 6454115.49],
 ['2025-08-31', 1495846378.06, 134147202.96, 6234935.04],
 ['2025-09-30', 1372496294.8, 123350083.25, 6018111.36],
 ['2025-10-31', 1

In [1]:
t = '''
计算日 期初本金余额 本金回收款 利息回收款 期末本金余额
2024/7/31 3,113,411,448.70 745,612,810.33 47,939,000.28 2,367,798,638.37
2024/8/31 2,367,798,638.37 323,388,412.67 10,730,951.30 2,044,410,225.70
2024/9/30 2,044,410,225.70 398,619,347.63 9,123,477.53 1,645,790,878.07
2024/10/31 1,645,790,878.07 365,337,137.05 7,225,380.82 1,280,453,741.02
2024/11/30 1,280,453,741.02 253,614,851.92 5,508,801.86 1,026,838,889.10
2024/12/31 1,026,838,889.10 108,198,298.10 4,389,691.63 918,640,591.00
2025/1/31 918,640,591.00 40,135,424.00 4,079,951.30 878,505,167.00
2025/2/28 878,505,167.00 31,105,997.00 3,878,036.01 847,399,170.00
2025/3/31 847,399,170.00 29,099,270.00 3,321,195.81 818,299,900.00
2025/4/30 818,299,900.00 1,939,400.00 3,543,482.20 816,360,500.00
2025/5/31 816,360,500.00 3,256,500.00 3,422,115.74 813,104,000.00
2025/6/30 813,104,000.00 26,345,000.00 3,505,866.83 786,759,000.00
2025/7/31 786,759,000.00 26,415,000.00 3,240,354.09 760,344,000.00
2025/8/31 760,344,000.00 86,639,000.00 3,226,298.92 673,705,000.00
2025/9/30 673,705,000.00 134,902,000.00 2,781,780.01 538,803,000.00
2025/10/31 538,803,000.00 91,050,000.00 2,247,260.06 447,753,000.00
2025/11/30 447,753,000.00 59,654,000.00 1,862,166.69 388,099,000.00
2025/12/31 388,099,000.00 29,370,000.00 1,643,368.01 358,729,000.00
2026/1/31 358,729,000.00 79,270,000.00 1,494,818.48 279,459,000.00
2026/2/28 279,459,000.00 82,509,000.00 1,167,989.69 196,950,000.00
2026/3/31 196,950,000.00 111,500,000.00 719,240.17 85,450,000.00
2026/4/30 85,450,000.00 85,450,000.00 316,185.12 0.00
'''

In [12]:
tz.pipe(t
       ,removeComma
       ,lambda x: x.strip().split("\n")
       ,splitSpace
       ,lambda xs: xs[1:]
       ,lambda xs: xs & lens.Each()[0].modify(lambda y:dateparser.parse(y))
        
       ,lambda xs: xs & lens.Each()[0].modify(lambda y: y.strftime("%Y-%m-%d")) 
       ,lambda xs: xs & lens.Each().Fork(lens[1],lens[2],lens[3],lens[4]).modify(float)
       ,lambda xs: list(transpose(xs))
       ,lambda xs: list(transpose([xs[0],xs[4],xs[2],xs[3]]))
       ,lambda xs: xs & lens.Each().modify(list)
       )

[['2024-07-31', 2367798638.37, 745612810.33, 47939000.28],
 ['2024-08-31', 2044410225.7, 323388412.67, 10730951.3],
 ['2024-09-30', 1645790878.07, 398619347.63, 9123477.53],
 ['2024-10-31', 1280453741.02, 365337137.05, 7225380.82],
 ['2024-11-30', 1026838889.1, 253614851.92, 5508801.86],
 ['2024-12-31', 918640591.0, 108198298.1, 4389691.63],
 ['2025-01-31', 878505167.0, 40135424.0, 4079951.3],
 ['2025-02-28', 847399170.0, 31105997.0, 3878036.01],
 ['2025-03-31', 818299900.0, 29099270.0, 3321195.81],
 ['2025-04-30', 816360500.0, 1939400.0, 3543482.2],
 ['2025-05-31', 813104000.0, 3256500.0, 3422115.74],
 ['2025-06-30', 786759000.0, 26345000.0, 3505866.83],
 ['2025-07-31', 760344000.0, 26415000.0, 3240354.09],
 ['2025-08-31', 673705000.0, 86639000.0, 3226298.92],
 ['2025-09-30', 538803000.0, 134902000.0, 2781780.01],
 ['2025-10-31', 447753000.0, 91050000.0, 2247260.06],
 ['2025-11-30', 388099000.0, 59654000.0, 1862166.69],
 ['2025-12-31', 358729000.0, 29370000.0, 1643368.01],
 ['2026-01-

In [19]:
t = '''
归集日期 期初剩余本金 回收本金 回收利息 期末剩余本金
2025-1-31 4,500,002,170.78 259,049,532.14 3,906,469.16 4,240,952,638.64
2025-2-28 4,240,952,638.64 255,975,141.22 7,170,295.04 3,984,977,497.42
2025-3-31 3,984,977,497.42 253,512,297.47 6,738,886.87 3,731,465,199.95
2025-4-30 3,731,465,199.95 249,354,135.07 6,311,620.99 3,482,111,064.88
2025-5-31 3,482,111,064.88 246,445,969.37 5,892,528.78 3,235,665,095.51
2025-6-30 3,235,665,095.51 235,420,257.22 5,478,946.39 3,000,244,838.29
2025-7-31 3,000,244,838.29 219,044,314.97 5,096,929.93 2,781,200,523.32
2025-8-31 2,781,200,523.32 204,441,807.63 4,760,854.25 2,576,758,715.69
2025-9-30 2,576,758,715.69 189,976,471.01 4,467,757.41 2,386,782,244.68
2025-10-31 2,386,782,244.68 179,361,307.49 4,212,038.66 2,207,420,937.19
2025-11-30 2,207,420,937.19 166,449,115.54 3,993,603.02 2,040,971,821.65
2025-12-31 2,040,971,821.65 162,393,655.56 3,812,075.68 1,878,578,166.09
2026-1-31 1,878,578,166.09 159,005,562.58 3,633,167.05 1,719,572,603.51
2026-2-28 1,719,572,603.51 156,441,592.27 3,457,647.06 1,563,131,011.24
2026-3-31 1,563,131,011.24 154,234,525.48 3,284,822.85 1,408,896,485.76
2026-4-30 1,408,896,485.76 152,766,137.57 3,113,650.99 1,256,130,348.19
2026-5-31 1,256,130,348.19 151,254,750.26 2,944,314.36 1,104,875,597.93
2026-6-30 1,104,875,597.93 149,503,729.43 2,776,157.65 955,371,868.50
2026-7-31 955,371,868.50 148,438,620.70 2,610,217.06 806,933,247.80
2026-8-31 806,933,247.80 146,786,652.35 2,446,812.98 660,146,595.45
2026-9-30 660,146,595.45 143,898,001.17 2,286,958.63 516,248,594.28
2026-10-31 516,248,594.28 117,485,521.28 2,137,322.04 398,763,073.00
2026-11-30 398,763,073.00 68,948,619.25 2,001,110.26 329,814,453.75
2026-12-31 329,814,453.75 20,556,658.98 1,883,092.34 309,257,794.77
2027-1-31 309,257,794.77 18,948,844.97 1,779,189.90 290,308,949.80
2027-2-28 290,308,949.80 17,075,556.96 1,684,821.60 273,233,392.84
2027-3-31 273,233,392.84 16,347,942.12 1,601,829.41 256,885,450.72
2027-4-30 256,885,450.72 15,455,731.58 1,523,291.30 241,429,719.14
2027-5-31 241,429,719.14 14,815,542.52 1,450,155.23 226,614,176.62
2027-6-30 226,614,176.62 14,789,853.69 1,380,960.16 211,824,322.93
2027-7-31 211,824,322.93 14,772,864.21 1,311,710.29 197,051,458.72
2027-8-31 197,051,458.72 14,802,055.41 1,242,424.00 182,249,403.31
2027-9-30 182,249,403.31 14,750,189.90 1,172,814.43 167,499,213.41
2027-10-31 167,499,213.41 13,450,677.09 1,103,325.23 154,048,536.32
2027-11-30 154,048,536.32 10,112,511.82 1,035,798.31 143,936,024.50
2027-12-31 143,936,024.50 6,319,311.18 980,283.55 137,616,713.32
2028-1-31 137,616,713.32 6,305,406.85 938,403.97 131,311,306.47
2028-2-29 131,311,306.47 6,318,440.05 896,581.76 124,992,866.42
2028-3-31 124,992,866.42 6,337,508.38 854,647.63 118,655,358.04
2028-4-30 118,655,358.04 6,362,106.61 812,547.41 112,293,251.43
2028-5-31 112,293,251.43 6,379,561.27 770,249.91 105,913,690.16
2028-6-30 105,913,690.16 6,408,996.09 727,769.89 99,504,694.07
2028-7-31 99,504,694.07 6,447,978.56 685,062.82 93,056,715.51
2028-8-31 93,056,715.51 6,478,498.59 642,066.65 86,578,216.92
2028-9-30 86,578,216.92 6,509,407.28 598,832.54 80,068,809.64
2028-10-31 80,068,809.64 6,474,963.95 555,360.46 73,593,845.69
2028-11-30 73,593,845.69 6,124,217.29 512,080.66 67,469,628.40
2028-12-31 67,469,628.40 5,840,716.47 469,914.33 61,628,911.93
2029-1-31 61,628,911.93 5,832,949.24 429,354.80 55,795,962.69
2029-2-28 55,795,962.69 5,822,629.87 388,840.31 49,973,332.82
2029-3-31 49,973,332.82 5,822,958.72 348,357.49 44,150,374.10
2029-4-30 44,150,374.10 5,844,580.64 307,847.55 38,305,793.46
2029-5-31 38,305,793.46 5,874,769.39 267,167.26 32,431,024.07
2029-6-30 32,431,024.07 5,914,197.51 226,267.27 26,516,826.56
2029-7-31 26,516,826.56 5,947,553.71 185,084.43 20,569,272.85
2029-8-31 20,569,272.85 5,984,223.97 143,657.59 14,585,048.88
2029-9-30 14,585,048.88 6,020,712.76 101,968.97 8,564,336.12
2029-10-31 8,564,336.12 5,598,449.80 60,029.99 2,965,886.32
2029-11-30 2,965,886.32 2,965,886.32 21,021.36 0.0
'''

In [20]:
## JSD

xx = tz.pipe(t
       #,lambda x: x.replace("-","0.0")
       ,removeComma
       ,lambda x: x.strip().split("\n")
       
       ,splitSpace
       ,lambda xs: xs[1:]
       ,lambda xs: xs & lens.Each()[0].modify(lambda y:dateparser.parse(y))
        
       ,lambda xs: xs & lens.Each()[0].modify(lambda y: y.strftime("%Y-%m-%d")) 
       ,lambda xs: xs & lens.Each().Fork(lens[1],lens[2],lens[3],lens[4]).modify(float)
       ,lambda xs: list(transpose(xs))
       ,lambda xs: list(transpose([xs[0],xs[4],xs[2],xs[3]]))
       ,lambda xs: xs & lens.Each().modify(list)
       )

xx

[['2025-01-31', 4240952638.64, 259049532.14, 3906469.16],
 ['2025-02-28', 3984977497.42, 255975141.22, 7170295.04],
 ['2025-03-31', 3731465199.95, 253512297.47, 6738886.87],
 ['2025-04-30', 3482111064.88, 249354135.07, 6311620.99],
 ['2025-05-31', 3235665095.51, 246445969.37, 5892528.78],
 ['2025-06-30', 3000244838.29, 235420257.22, 5478946.39],
 ['2025-07-31', 2781200523.32, 219044314.97, 5096929.93],
 ['2025-08-31', 2576758715.69, 204441807.63, 4760854.25],
 ['2025-09-30', 2386782244.68, 189976471.01, 4467757.41],
 ['2025-10-31', 2207420937.19, 179361307.49, 4212038.66],
 ['2025-11-30', 2040971821.65, 166449115.54, 3993603.02],
 ['2025-12-31', 1878578166.09, 162393655.56, 3812075.68],
 ['2026-01-31', 1719572603.51, 159005562.58, 3633167.05],
 ['2026-02-28', 1563131011.24, 156441592.27, 3457647.06],
 ['2026-03-31', 1408896485.76, 154234525.48, 3284822.85],
 ['2026-04-30', 1256130348.19, 152766137.57, 3113650.99],
 ['2026-05-31', 1104875597.93, 151254750.26, 2944314.36],
 ['2026-06-30'

In [25]:
## JSD PAC
t = '''
支付日 还款金额(元) 目标余额(元)
2025-04-26 1,600,000,000.00
2025-05-26 1,600,000,000.00
2025-06-26 1,600,000,000.00
2025-07-26 1,376,000,000.00
2025-08-26 1,152,000,000.00
2025-09-26 928,000,000.00
2025-10-26 704,000,000.00
2025-11-26 480,000,000.00
2025-12-26 360,000,000.00
2026-01-26 240,000,000.00
2026-02-26 120,000,000.00
2026-03-26 0.00
'''

In [27]:
xx = tz.pipe(t
       #,lambda x: x.replace("-","0.0")
       ,removeComma
       ,lambda x: x.strip().split("\n")
       
       ,splitSpace
       ,lambda xs: xs[1:]
       #,lambda xs: xs & lens.Each()[0].modify(lambda y:dateparser.parse(y))
        
       #,lambda xs: xs & lens.Each()[0].modify(lambda y: y.strftime("%Y-%m-%d")) 
       ,lambda xs: xs & lens.Each().Fork(lens[1]).modify(float)
       #,lambda xs: list(transpose(xs))
       #,lambda xs: list(transpose([xs[0],xs[4],xs[2],xs[3]]))
       #,lambda xs: xs & lens.Each().modify(list)
       )

xx

[['2025-04-26', 1600000000.0],
 ['2025-05-26', 1600000000.0],
 ['2025-06-26', 1600000000.0],
 ['2025-07-26', 1376000000.0],
 ['2025-08-26', 1152000000.0],
 ['2025-09-26', 928000000.0],
 ['2025-10-26', 704000000.0],
 ['2025-11-26', 480000000.0],
 ['2025-12-26', 360000000.0],
 ['2026-01-26', 240000000.0],
 ['2026-02-26', 120000000.0],
 ['2026-03-26', 0.0]]